<img src="images/kiksmeisedwengougent.png" alt="Banner" width="1100"/>

<div>
    <font color=#690027 markdown="1">   
<h1>DEMARCATION LINE STOMATA ON SUNLIT AND SHADED LEAVES</h1>    </font>
</div>

<div class="alert alert-box alert-success">
In this notebook, you will determine a straight line that separates the sunned and shaded leaves (approximately). For this, you will use machine learning techniques, such as gradient descent.</div>

Krappa or crabwood is a rapidly growing tree species that is common in the Amazon region. Mature specimens can have a diameter of more than a meter and can be more than 40 meters high. <br>The high-quality wood is used for making furniture, flooring, masts ... A fever-reducing agent is extracted from the bark. Oil for medicinal applications, including the treatment of skin diseases and tetanus, and as a repellent for insects, is produced from the seeds.

<table><tr>
<td> <img src="images/andirobaamazonica.jpg" alt="Drawing" width="200"/></td>
<td> <img src="images/crabwoodtree.jpg" alt="Drawing" width="236"/> </td>
</tr></table>

<center>
Photos: Mauroguanandi [Public domain] [2] and P. S. Sena [CC BY-SA 4.0] [3].</center>

Because some climate models predict a rise in temperature and a decrease in rainfall in the coming decades, it is important to know how these trees adapt to changing conditions. <br>Scientists Camargo and Marenco conducted research in the Amazon rainforest [1].<br>In addition to the influence of seasonal rainfall, they also examined stomatal characteristics of leaves under sunny and under shaded conditions. <br> For this, a number of plants, grown in the shade, were moved to full sunlight for 60 days. Another group of plants was kept in the shade. <br> The characteristics of the stomata were measured on prints of the leaves made with transparent nail polish.

### Import necessary modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from matplotlib import animation
from IPython.display import HTML

<div>
    <font color=#690027 markdown="1">   
<h2>1. Reading the data</h2>    </font>
</div>

Read the dataset using the `pandas` module.

In [ ]:
stomata = pd.read_csv("data/schaduwzon.csv", header="infer")  # table to be read has a heading

<div>
    <font color=#690027 markdown="1">   
<h2>2. Step by step in search of the dividing line</h2>    </font>
</div>

Just like the researchers, you plot the stomatal density against the stomatal length.

Just like in the previous notebook, the standardized data is used for x$_{1}$ and x$_{2}$.

In [ ]:
# label data
x1 = stomata["stomatal length"]          # feature: length
x2 = stomata["stomatal density"]       # attribute: density
x1 = np.array(x1)          # feature: length
x2 = np.array(x2)          # feature: density
# standardize
x1 = (x1 - np.mean(x1)) / np.std(x1)
x2 = (x2 - np.mean(x2)) / np.std(x2)
# labels
y = stomata["milieu"]            # labels: second column of the original table
y = np.array(y)
y = np.where(y == "zon", 1, 0)     # make labels numeric, sun:1, shadow:0

In [ ]:
X = np.stack((x1, x2), axis = 1)    # correct formatone_colum
n = np.ones((X.shape[0],1))
X = np.concatenate((one_column, X), axis = 1)   # Add 1 at every point

In [ ]:
# training set with input X(x1, x2) and output y
print(X)
print(y)

<div>
    <font color=#690027 markdown="1">   
<h3>2.1 Structure of the algorithm</h3>    </font>
</div>

Such a dividing line is searched for with an algorithm. Here you can see how such an algorithm is structured.

<div class="alert alert-box alert-info">
The ML system is a neural network without a hidden layer and with the activation function being the sigmoid function.<br> The error function used is binary cross entropy.<br>To find a line that separates the two classes, the ML system starts with a randomly chosen line. This is done by randomly choosing the slope and the y-intercept of this line.<br>The system is <em>trained</em> with the training set (the inputs and the corresponding labels): for each point of the training set, it is determined how much the error is. The coefficients in the equation of the line are adjusted until the error is minimal. <br>The entire training set is run through a number of times. Such a time is called an <em>epoch</em>. The system <em>learns</em> during these <em>attempts ('epochs')</em>.</div>

The neural network first makes a linear combination of the input with the weights.<br> The **activation function** then works on this result. In this neural network, it's *sigmoid*. For each data point, the sigmoid function returns a value between 0 and 1. This value indicates how certain the system is that the point belongs to the class with label 1.

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def predict(features, weights):
    "The prediction is a value that indicates how certain the point belongs to the class with label 1."    
    
    z = np.dot(features, weights.T)
    prediction = sigmoid(z)    

    return prediction

The system should be able to calculate the error after each epoch. <br>For this purpose, the residual $y-\hat{y}$ is calculated for each point. Here, $y$ is the given y-value and $\hat{y}$ is the predicted value, i.e., the value obtained by substituting the given x-value into the equation of the line.<br> The squares of the residuals are added together. This sum divided by the number of data points is the error sought.

In [ ]:
def bc(features, labels, weights):
    """Calculate error binary crossentropy."""    
    n = len(y)            # number of points
    predictions = predict(features, weights)            # current prediction    

    #Take the error when label=1
    class1_cost = - labels * np.log(predictions)
    #Take the error when label=0
    class2_cost = (1 - labels) * np.log(1-predictions)    
    #Take the sum of both costs
    cost = class1_cost + class2_cost
    #Take the average cost
    cost = cost.mean()
    
    return cost

# def loss(h, y):
#    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

In [ ]:
def gradient_descent(features, labels, weights, eta):
    """Adjustment of parameters q and m after completed epoch with learning rate eta."""    
    
    n = len(labels)                                 # number of points is number of values in list of labels y
    predictions = predict(features, weights)        # view current predictions     

    # Transpose features X 
    # So we can multiply with the matrix.# Returns a (3,1) matrix (predictions - labels)
    
    # calculating the partial derivatives
    gradient = np.dot(features.T, (predictions - labels))    
    gradient = gradient / n    
    
    # adjust values weights    
    weights = weights - eta *gradient  
    # return customized weights
    return weights

<div>
    <font color=#690027 markdown="1">   
<h3>2.2 Testing the gradient descent algorithm for multiple epochs</h3>    </font>
</div>

Take (0; 1; 0.2) as the initial value for the *weights*. Perform gradient descent for 200 epochs with a learning rate of 0.01, displaying the adjustments to the *weights* and the error after each *epoch*.

In [ ]:
# testing algorithm
w = np.array([0, 1, 0.2])
eta = 0.01

for j in range(200):
    fout = bc(X,y,w)                       # calculate binary crossentropy after each epoch
    print(j, w, fout)                      # display weights and error values after each epoch
    w = gradient_descent(X, y, w, eta)     # adjust weights values after each epoch    

print("The line intersects the y-axis at: %.3f" % (-w[0]/w[2]))
print("The line has as slope: %.3f" % (-w[1]/w[2]))
print("Binary crossentropy for the line w.r.t. the data: %.4f" % fout)

In the example, you can see that the number of epochs will help determine how accurately the dividing line is determined. The line that was found after e.g. 20 epochs is still very far from the intended dividing line. Also look at how the error develops; as long as it continues to decrease in absolute value, it has not been minimized yet, the system is then *underfit*. Apparently the error does increase again. Perhaps the *learning rate* is too large.

<div>
    <font color=#690027 markdown="1">   
<h3>2.3 How do the error and the position of the line change during the process?</h3>    </font>
</div>

In [ ]:
def gradient_descent_process(features, labels, weights, eta, epochs):
    """Go through the process and gradually make lists of q, m and error."""
    list_error = [bc(features, labels, weights)]      # Declare and initialize error list
    list_weights = [weights]                          # declare and initialize list of weights    

    # Fill lists for each epoch
    for i in range(epochs):
        weights = gradient_descent(features, labels, weights, eta)    # modified parameters after epoch
        error = bc(features, labels, weights)                        # cost after epoch
        list_weights.append(weights)                           # add adjusted q
        list_fout.append(fout)                           # add this error

    return [list_weights, list_error]

Go through the process for chosen initial values for the weights, chosen *learning rate* and chosen number of *epochs*.

In [ ]:
# initialization of the weights
w = np.array([0, 1, 0.2])

# recording the number of epochs and learning rate èta
eta = 0.01
epochs = 1000

# running linear regression algorithm for choice of weights, èta and epochs
list_weights, list_error = gradient_descent_process(X, y, w, eta, epochs)

# dividing line
print ("Passage y-axis: %.3f" % (-list_weights[-1][0]/list_weights[-1][2]))
print ("Slope: %.3f" % (-list_weights[-1][1]/list_weights[-1][2]))

# average square deviation regression line
print ("Minimized error: %.4f" %  list_error[-1])

An animation:

In [ ]:
# all straight lines
xcoord = np.linspace(-2, 2, 30)

ycoord = []
for j in range(epochs):    
    y_r = (-list_weights[j][1]/list_weights[j][2]) * xcoord + (-list_weights[j][0]/list_weights[j][2]) # calculate y-value from all x's from xcoord for the respective line    
    ycoord.append(y_r)
ycoord = np.array(ycoord)    # type casting

# initialize plot window
fig, ax = plt.subplots()
line, = ax.plot(xcoord, ycoord[0], color="green")   # plot line

ax.axis([x1.min()-1,x1.max()+1,x2.min()-1,x2.max()+1])  # axis range
plt.title("Amazon sun-shadow standardized")
plt.xlabel("stomata length")              # xlabel provides a description on the x-axis
plt.ylabel("stomatal density")         # ylabel gives a description on the y-axis
plt.scatter(x1[:25], x2[:25], color="lightgreen", marker="o", label="sun")      # sun's first 25 (label 1)
plt.scatter(x1[25:], x2[25:], color="darkgreen", marker="o", label="shadow")   # shadow are the next 25 (label 0)

def animate(i):
    line.set_ydata(ycoord[i])    # update the equation of the line

plt.close()  # to temporarily close the plot window, only animation screen needed

anim = animation.FuncAnimation(fig, animate, repeat=False, frames=len(ycoord))    

HTML(anim.to_jshtml())

In [ ]:
# graph evolution error
plt.figure(figsize=(10,8))
plt.plot(list_error)
plt.xlabel('epoch')
plt.ylabel('binary cross entropy')
plt.title('Evolution of the error')
plt.show()

Experiment with the *learning rate* and the number of *epochs*.

<div class="alert alert-box alert-warning">
In the following notebook, you perform the classification using an existing Python module, namely scikit-learn.</div>

<div>
<h2>Reference List</h2></div>

[1] Camargo, Miguel Angelo Branco, & Marenco, Ricardo Antonio. (2012). Growth, leaf and stomatal traits of crabwood (Carapa guianensis Aubl.)<br> &nbsp; &nbsp; &nbsp; &nbsp; in central Amazonia. Revista Árvore, 36(1), 07-16. https://dx.doi.org/10.1590/S0100-67622012000100002 and via e-mail.<br>[2] Mauroguanandi [Public domain]. https://commons.wikimedia.org/wiki/File:Andirobaamazonica.jpg. <br> &nbsp; &nbsp; &nbsp; &nbsp; Consulted on August 13, 2019 via Wikimedia Commons. <br>[3] Sena, P. S. https://commons.wikimedia.org/wiki/File:Crabwood_tree.JPG. [CC BY-SA 4.0] Accessed on August 13, 2019 via Wikimedia Commons.

<div>
<h2>With support from</h2></div>

<img src="images/kikssteun.png" alt="Banner" width="1100"/>

<img src="images/cclic.png" alt="Banner" align="left" width="100"/><br><br>
Notebook KIKS, see <a href="http://www.aiopschool.be">AI At School</a>, by F. wyffels & N. Gesquière, is licensed under a <a href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.